In [27]:
%matplotlib inline
from PIL import Image
import numpy
import os
import re
import random
from resizeimage import resizeimage
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.decomposition import PCA
import copy
import matplotlib.image as mpimg
from sklearn.decomposition import RandomizedPCA

In [59]:
# resize images
# resizeImages('../data/yalefaces/', 'subject01')
def resizeImagesToDest(source, dest, prefix, height, noiseAlgo=None):    
    for root, dirs, files in os.walk(source):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(os.path.join(source, name))
                (origWidth, origHeight) = img.size
                if (height == origHeight):
                    resizedImg = img
                else:
                    resizedImg = resizeimage.resize_height(img, height)
                    
                (width, height) = resizedImg.size
                if noiseAlgo:
                    noisyImg = noiseAlgo(numpy.asarray(resizedImg))
                    Image.fromarray(numpy.uint8(noisyImg.reshape(height,width))).save(dest+name)
                else:
                    resizedImg.save(dest+name)

def resizeImages(source, prefix):
    resizeImagesToDest(source, 'yalefaces_resized/', prefix, 50)
                
# images into array: http://stackoverflow.com/questions/13550376/pil-image-to-array-numpy-array-to-array-python
def readImagesIntoMatrix(path, prefix):
    images = None
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)                
                (width, height) = img.size
                imgArr = numpy.asarray(img).reshape(height * width)
                if images is not None:
                    images = numpy.vstack([images, imgArr])
                else:
                    images = imgArr
    return images

def saveImages(path, images, width, height):
    for i in range(0, len(images)):
        Image.fromarray(numpy.uint8(images[i]).reshape(height, width)).save(path+str(i)+'.gif')
        
def addGaussianNoise(image):
    noisyImage = copy.deepcopy(image)
    mu = noisyImage.mean()
    std = noisyImage.std()
    noise = 0.2 * numpy.random.normal(mu, std, noisyImage.shape)
    for i in range(len(image)):
        for j in range(len(image[0])):
            val = image[i][j]
            if val + noise[i][j] >= 255:
                noisyImage[i][j] = 255
            elif val + noise[i][j] <= 0:
                noisyImage[i][j] = 0
            else:
                noisyImage[i][j] = val + noise[i][j] 
    
    return noisyImage  
    
def addSparseNoise(image):
    noisyImage = copy.deepcopy(image)
    for i in range(len(image)):
        for j in range(len(image[0])):
            rand = random.random()
            if (rand <= 0.01):
                noisyImage[i][j] = 0
    
    return noisyImage

In [92]:
resizeImages('yalefaces/', 'yale.*')
imageList = readImagesIntoMatrix('yalefaces_resized/', '.*').T

print("Images %s" % imageList)

Image.fromarray(numpy.uint8(imageList.T[0].reshape(50,44))).save("secondyaletest/testsaveimage1.gif")

Images [[155   4 135 ..., 161  84   5]
 [152   4 135 ..., 162  84   5]
 [136   4 120 ..., 166  88   5]
 ..., 
 [ 11 145   6 ...,   6   4 163]
 [ 11 141   6 ...,   6   6 178]
 [ 12 136   7 ...,  13   8 187]]


In [64]:
(width, height) = 44, 50
# average Image
#print('Num images %s' % (imageList.shape,))
avgImg = imageList.mean(axis=1)
print('Average image %s' % (avgImg))
normalizedImageList = imageList - avgImg.reshape(width*height,1)*numpy.ones((1,2414))
Image.fromarray(numpy.uint8(avgImg.reshape(height,width))).save('secondyaletest/avgImg.gif')

Average image [ 59.02526926  62.63835957  64.9908865  ...,  49.28583264  46.52734051
  40.11267606]


In [95]:
def normalize_face(vector):
    minValue = numpy.amin(vector)
    maxValue = numpy.amax(vector)
    scale = (255./(maxValue-minValue))
    return (vector - minValue)*scale

# Fit with PCA for 90% of variance
pca = PCA(0.9).fit_transform(normalizedImageList)
print('With PCA there were %s eigenfaces that captures 90pct of the variance' % len(pca.T))

for i in range(len(pca.T)):
    eigv = pca[:,i].reshape(1,width * height).T
    eigenface = normalize_face(eigv)
    Image.fromarray(numpy.uint8(eigenface.reshape(height,width))).save('secondyaletest/eigenface%s.gif' % i)

With PCA there were 33 eigenfaces that captures 90pct of the variance


In [76]:
from numpy.linalg import norm, svd
from scipy.io import loadmat, savemat

def inexact_augmented_lagrange_multiplier(X, lmbda=.01, tol=1e-3,
                                          maxiter=100, verbose=True):
    """
    Inexact Augmented Lagrange Multiplier
    """
    Y = X
    norm_two = norm(Y.ravel(), 2)
    norm_inf = norm(Y.ravel(), numpy.inf) / lmbda
    dual_norm = numpy.max([norm_two, norm_inf])
    Y = Y / dual_norm
    A = numpy.zeros(Y.shape)
    E = numpy.zeros(Y.shape)
    dnorm = norm(X, 'fro')
    mu = 1.25 / norm_two
    rho = 1.5
    sv = 10.
    n = Y.shape[0]
    itr = 0
    while True:
        if verbose:
            print('Iterations %s' % itr)
        Eraw = X - A + (1 / mu) * Y
        Eupdate = numpy.maximum(Eraw - lmbda / mu, 0) + numpy.minimum(Eraw + lmbda / mu, 0)
        U, S, V = svd(X - Eupdate + (1 / mu) * Y, full_matrices=False)
        svp = (S > 1 / mu).shape[0]
        if svp < sv:
            sv = numpy.min([svp + 1, n])
        else:
            sv = numpy.min([svp + round(.05 * n), n])
        Aupdate = numpy.dot(numpy.dot(U[:, :svp], numpy.diag(S[:svp] - 1 / mu)), V[:svp, :])
        A = Aupdate
        E = Eupdate
        Z = X - A - E
        Y = Y + mu * Z
        mu = numpy.min([mu * rho, mu * 1e7])
        itr += 1
        if ((norm(Z, 'fro') / dnorm) < tol) or (itr >= maxiter):
            break
    if verbose:
        print("Finished at iteration %d" % (itr))  
    return A

In [77]:
 = inexact_augmented_lagrange_multiplier(normalizedImageList)

Iterations 0
Iterations 1
Iterations 2
Iterations 3
Iterations 4
Iterations 5
Iterations 6
Iterations 7
Iterations 8
Iterations 9
Iterations 10
Iterations 11
Iterations 12
Iterations 13
Iterations 14
Iterations 15
Iterations 16
Iterations 17
Iterations 18
Iterations 19
Iterations 20
Iterations 21
Iterations 22
Iterations 23
Iterations 24
Iterations 25
Iterations 26
Iterations 27
Iterations 28
Finished at iteration 29


In [91]:
Image.fromarray(numpy.uint8((A.T[0] + avgImg).reshape(50,44))).save("secondyaletest/testsaveimage1afterrpca.gif")



normalizedImageList is 135448.449117


In [97]:

# Fit with PCA for 90% of variance
pca = PCA(0.9).fit_transform(A)

print('After RPCA there were %s eigenfaces that captures 90pct of the variance' % len(pca.T))

for i in range(len(pca.T)):
    eigv = pca[:,i].reshape(1,width * height).T
    eigenface = normalize_face(eigv)
    Image.fromarray(numpy.uint8(eigenface.reshape(height,width))).save('secondyaletest/rpca-eigenface%s.gif' % i)

After RPCA there were 3 eigenfaces that captures 90pct of the variance
